This notebook demonstrates how to use the `HelmholtzSolver` class. First, the object has to be initiated. Then, the required coefficients can be computed using the `compute_coefficients` method of the class. The method returns:
- $R$
- $|R|^2$
- $T$
- $|T|^2$
- $A = 1 - |R|^2 - |T|^2$
where the coefficients are defined according to the asymptotic conditions: 

$$
E(z) \rightarrow 
\begin{cases}
    e^{ik_0z}+Re^{-ik_0z}, & z \rightarrow -\infty \\
    Te^{ik_0z}, & z \rightarrow +\infty
\end{cases}
$$

The slab structure is defined by providing a list coordinates of the interfaces and a list defining a piecewise dielectric permittivity. 

**Units**:
- $z$, $\lambda$: meters/micrometers, etc
- $\theta$: deg

In [1]:
from HelmholtzSolver import HelmholtzSolver
import numpy as np

def format_complex(z):
    # Разделяем действительную и мнимую части
    real = z.real
    imag = z.imag
    # Форматируем с фиксированной шириной и обязательным знаком
    return f"{real:+.4f}{imag:+.4f}j"

def print_coefficients(R, T):
    R_str = format_complex(R)
    T_str = format_complex(T)
    R_pow = f"{np.abs(R)**2:.4f}"
    T_pow = f"{np.abs(T)**2:.4f}"
    A = f"{1 - np.abs(R)**2 - np.abs(T)**2:.4f}"

    print(f"  R = {R_str:<16} |R|^2 = {R_pow:>8}")
    print(f"  T = {T_str:<16} |T|^2 = {T_pow:>8}")
    print(f"  A = {A:>7}")

### Example

In [2]:
solver_s = HelmholtzSolver(
    lambda0=15,
    boundaries=[0, 0.5, 1.0, 1.5],
    epsilons=[1.5, -30 + 40j, 10 + 3j],
    theta=10,
    polarization="p"
)
R, T, A = solver_s.compute_coefficients()

print_coefficients(R, T)

  R = -0.6849-0.5138j  |R|^2 =   0.7330
  T = +0.1663+0.0100j  |T|^2 =   0.0277
  A =  0.2392


### Absorption

In [3]:
epsilon = 4 + 1j  
solver = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,  
)
R, T, A = solver.compute_coefficients()
assert A > 0, f"Ошибка: A = {A} (должно быть > 0)"
print(f"A = {A:.4f}")

A = 0.2314


### Normal incidence (s equals to p)

In [4]:
epsilon = 2.25
solver_s = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,
    polarization="s"
)
R_s, T_s, A_s = solver_s.compute_coefficients()

solver_p = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0.1],
    epsilons=[epsilon],
    theta=0,
    polarization="p"
)
R_p, T_p, A_p = solver_p.compute_coefficients()

assert np.isclose(abs(R_s), abs(R_p), atol=1e-6), f"Ошибка: |R_s|={abs(R_s)}, |R_p|={abs(R_p)}"

### No layers

In [5]:
epsilon = 4 + 1j  
solver = HelmholtzSolver(
    lambda0=1.0,
    boundaries=[0, 0],
    epsilons=[15],
    theta=0,  
)
R, T, A = solver.compute_coefficients()

assert np.isclose(abs(R), 0, atol=1e-6)